## Notebook for scraping wiki page for toronto data

In this notebook, I'm scraping the Toronto Postal Codes from a wiki page for a Coursera course (see link below). The following tasks are done:

- read the wiki page and load it into a BeautifulSoup object
- parse the BS object and extract the data from the html table while writing it to a first dataframe
- then iterate over that df and copy the data while grouping the neighborhoods over the boroughs

ref.: https://www.coursera.org/learn/applied-data-science-capstone


First, some basic settings:

In [1]:
# where to find the toronto data
wiki_url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# column names for target dataframe
mycolumns = ['PostalCode', 'Borough','Neighborhood']


Now import the libraries needed

In [2]:
##
## import libraries
##
import pandas as pd
import sys
import urllib
import urllib.request
from bs4 import BeautifulSoup
print('libaries imported')

libaries imported


## The scraping part

Load the html page

In [3]:
soup = BeautifulSoup(
       urllib.request.urlopen( wiki_url ).read(),
       'html.parser'
       )

## table is <table class="wikitable sortable">
mytable = soup.findAll('table',{'class':'wikitable'})[0]

# check first bytes
print( str(mytable)[0:111] )


<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>



#### Copy the data into a pandas df

- find all rows (tr)
- in each row, find the cells (td)
- extract the text and strip appending newlines
- if not 'Not assigned', copy to df

In [4]:
# create empty dataframe
df_raw = pd.DataFrame(columns=mycolumns)

## extract all rows
rows = mytable.find_all('tr')

## write to df
for row in rows:
    line = ''
    cells = row.find_all('td')
    
    # no cells in header (<th>!!)
    if cells:
        P = cells[0].get_text().strip()
        B = cells[1].get_text().strip()
        N = cells[2].get_text().strip()
        #print(P + '\t' + B + '\t' + N)
        
        # add to df_raw only if Borough is assigned
        if B != 'Not assigned':
            df_raw.loc[len(df_raw)] = [ P, B, N]

# show df_raw
df_raw.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Show the shape of the df

In [5]:
df_raw.shape

(212, 3)

Show number of unique postal codes

In [6]:
# unique() returns array; shape[0] is number of rows
df_raw['PostalCode'].unique().shape[0]

103

Extract the PostalCodes to iterate over in next step

In [7]:
TPCodes = df_raw['PostalCode'].unique()
TPCodes[0:5]

array(['M3A', 'M4A', 'M5A', 'M6A', 'M7A'], dtype=object)

## Main Loop

This is the main part where the heavy lifting is done:
- copying data for each postalCode
- aggregating the Neighborhoods

In [8]:
df_grouped = pd.DataFrame(columns=mycolumns)
df_grouped

for TPC in TPCodes:
    # filter for TPC
    df_temp = df_raw[df_raw['PostalCode'] == TPC]
    
    # set PostalCode & Borough; Borough is 1:1 to PostalCode and therefor unique here 
    P = TPC
    B = df_temp['Borough'].unique()[0]
    
    # aggregate Neighborhoods by using temp df
    NN   = ''  # empty Neighborhood string
    cnt  = 1   # set counter for rows in df
    # get number of rows
    num_rows = df_temp.shape[0]
    
    # iterate over temp df
    for index,row in df_temp.iterrows():
        NN = NN + row['Neighborhood']
        if cnt < num_rows:
            NN  += ','
            cnt += 1
    
    # if no neighborhood is assigned, take over boroughs name
    if NN == 'Not assigned':
        NN = B
    
    # add data to new df
    df_grouped.loc[len(df_grouped)] = [ P, B, NN]
        
#     
df_grouped


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Finally, the shape of the generated dataframe:

In [9]:
df_grouped.shape

(103, 3)

So far for getting the data.

## Step2: enriching with Geo Coords

In this par we add longitude  and lattitude to the dataframes rows

### install the geocoder library

In [10]:

!pip install geocoder
import geocoder # import geocoder

print('\n**** installed geocoder ****\n')

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,

### define a function for retrieving long&lattm

In [11]:

def fn_get_lattlong(postal_code):

  # initialize your variable to None
  lat_lng_coords = None

  # loop until you get the coordinates
  while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng
    sys.stdout.write('.')

  latitude  = lat_lng_coords[0]
  longitude = lat_lng_coords[1]
  return latitude,longitude

print('defined function')

defined function


#### The next step _would_ get the coordinates from Google but GeoCoder always returned _None_ in m case

#### Instead,  I'm using the CSV file provided by Coursera

Prepare CSV working

In [12]:
import csv
csv_url ='https://cocl.us/Geospatial_data'

Load data into separate dataframe

In [ ]:
df_csv = pd.read_csv( csv_url )
df_csv.head()

#### add new column and fill with default

In [ ]:

df_grouped['Latitude']  = 0.0
df_grouped['Longitude'] = 0.0


Iterate over grouped dataframe and copy long/latt from csv data



In [ ]:

for index,row in df_grouped.iterrows():
    pc = row['PostalCode']
    # show PostalCode we're working on to indicate progress
    #sys.stdout.write( pc + ' ' )
    
    # get coords and save to df_grouped
    latt = df_csv.loc[ df_csv['Postal Code'] == pc ]['Latitude'].values[0]
    long = df_csv.loc[ df_csv['Postal Code'] == pc ]['Longitude'].values[0]
    
    # set values in grouped df
    df_grouped.loc[ df_grouped['PostalCode'] == pc, 'Latitude' ]  = latt
    df_grouped.loc[ df_grouped['PostalCode'] == pc, 'Longitude' ] = long




## Here we are:

In [ ]:
df_grouped.head()